# Doc Q&A Demo
This notebook contains an example of Doc Q&A, where a user can upload a document and ask questions about it. The pipeline will take the following steps:
1. Parse the document in text format
2. Chunk the text
3. Embed each chunk
4. Index chunks and store in an in-memory vector database to allow semantic search

The example is built with the llama-index library.

References: https://docs.llamaindex.ai/en/stable/understanding/putting_it_all_together/q_and_a/#qa-patterns

In [23]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import TextNode

In [24]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Load the data

In [2]:
# Load all text document from the folder docs/
documents = SimpleDirectoryReader("docs").load_data()

In [3]:
print(type(documents))
print(len(documents))

<class 'list'>
1


In [4]:
doc_0 = documents[0]
print(type(doc_0))
print(doc_0.dict().keys())
print(doc_0.metadata)

<class 'llama_index.core.schema.Document'>
dict_keys(['id_', 'embedding', 'metadata', 'excluded_embed_metadata_keys', 'excluded_llm_metadata_keys', 'relationships', 'text', 'start_char_idx', 'end_char_idx', 'text_template', 'metadata_template', 'metadata_seperator', 'class_name'])
{'file_path': '/home/experiments/docs/state_of_the_union.txt', 'file_name': 'state_of_the_union.txt', 'file_type': 'text/plain', 'file_size': 39027, 'creation_date': '2023-05-10', 'last_modified_date': '2023-05-10'}


In [17]:
# Print all properties and methods of a Document object
print(dir(doc_0))

['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__', '_abc_impl', '_calculate_keys', '_compat_fields', '_copy_and_set_values', '_decompose_class', '_enforce_dict_if_root', '_get_value', '_init

In [16]:
# View the first few lines of object doc_0
print(doc_0.text[:1000])

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

Groups of citizens blocking tanks with their bodies. Every

# Index the documents
The basic LlamaIndex example uses the one-line command `VectorStoreIndex.from_documents` to index/chunk/embed all the documents. It wouldn't work in my case though, as I would keep running into a `RateLimitError`. The error message pointed toward my OpenAI account. After some [digging](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/embeddings/utils.py#L31), I could confirm that LlamaIndex's default embedding model is OpenAI's; which would fail in my case as my account is empty. Also, since I want to use open-source solutions in that example, I need to use a different approach.

The solution is to define explicitly the embedding model that I want to use. And to do that in LlamaIndex, we need to [use the ingestion pipeline](https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/#using-the-ingestion-pipeline-to-create-nodes). LlamaIndex even has a nice [tutorial](https://docs.llamaindex.ai/en/stable/examples/low_level/oss_ingestion_retrieval/) and how to set it up with the sentence embedding model from HuggingFace's transformer library, which is exactly what I was hoping to do.

## Split the document

In this example, we're using `SentenceSplitter` which is a pretty basic type of text splitter, only making sure to not break sentences (see [documentation](https://docs.llamaindex.ai/en/stable/api_reference/node_parsers/sentence_splitter/)). 

It would be interesting to play with more complex splitter like `SemanticSplitter` which attempts to build chunks containing text that is semantically related ([doc](https://docs.llamaindex.ai/en/stable/api_reference/node_parsers/semantic_splitter/)). That imo makes a lot of sense and is an idea I was playing with in the past. It'd be interesting to see how it works.

In [54]:
SentenceSplitter?

Init signature:
SentenceSplitter(
    separator: str = ' ',
    chunk_size: int = 1024,
    chunk_overlap: int = 200,
    tokenizer: Optional[Callable] = None,
    paragraph_separator: str = '\n\n\n',
    chunking_tokenizer_fn: Optional[Callable[[str], List[str]]] = None,
    secondary_chunking_regex: str = '[^,.;。？！]+[,.;。？！]?',
    callback_manager: Optional[llama_index.core.callbacks.base.CallbackManager] = None,
    include_metadata: bool = True,
    include_prev_next_rel: bool = True,
    id_func: Optional[Callable[[int, llama_index.core.schema.Document], str]] = None,
) -> None
Docstring:     
Parse text with a preference for complete sentences.

In general, this class tries to keep sentences and paragraphs together. Therefore
compared to the original TokenTextSplitter, there are less likely to be
hanging sentences or parts of sentences at the end of the node chunk.
Init docstring: Initialize with parameters.
File:           /usr/local/lib/python3.11/site-packages/llama_index/cor

In [25]:
chunk_size = 1024
text_parser = SentenceSplitter(
    chunk_size=chunk_size,
    # separator=" ",
)

In [26]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

In [47]:
print(f"All documents were split into a total of {len(text_chunks)} chunks.")
print(f"The total lenght of all chunks is {sum([len(ch) for ch in text_chunks])}" + 
      f" compared to {len(doc_0.text)} for the original document." +
      f" It is therefore likely that there is overlap across the chunks."
     )

All documents were split into a total of 10 chunks.
The total lenght of all chunks is 46228 compared to 38539 for the original document. It is therefore likely that there is overlap across the chunks.


In [48]:
chunk_expl = text_chunks[4]
print(type(chunk_expl), len(chunk_expl))
chunk_expl[:500]

<class 'str'> 4532


'More jobs where you can earn a good living in America. \n\nAnd instead of relying on foreign supply chains, let’s make it in America. \n\nEconomists call it “increasing the productive capacity of our economy.” \n\nI call it building a better America. \n\nMy plan to fight inflation will lower your costs and lower the deficit. \n\n17 Nobel laureates in economics say my plan will ease long-term inflationary pressures. Top business leaders and most Americans support my plan. And here’s the plan: \n\nFirst – cut'

In [53]:
# Do the chunks overlap? Check whether the first sentence of the second chunk can be found in the first chunk
text_chunks[1].split(".")[0] in text_chunks[0]

True

In [55]:
# TODO: Calculate by how much (how many characters) each consecutive chunks overlap

In [40]:
print(text_chunks[0][1000:1100])
print(doc_0.text[1000:1100])

one from students to retirees teachers turned soldiers defending their homeland. 

In this struggle 
one from students to retirees teachers turned soldiers defending their homeland. 

In this struggle 


In [43]:
len_00 = len(text_chunks[0])
print(text_chunks[0][len_00-200:len_00])


l not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.


In [44]:
print(doc_0.text[len_00-200:len_00+100])

l not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to 


## Construct a Node for each text chunk

Nodes are a concept specific to LlamaIndex (afaik). They are chunks of documents (text, image, audio,...) augmented with metadata and relational information (for more, see the [LlamaIndex documentation](https://docs.llamaindex.ai/en/stable/module_guides/loading/documents_and_nodes/usage_nodes/)).

In [ ]:
nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(text=text_chunk,)
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

## Embed each Node

Embeddings of each Node are added to the Node in the form of a property (`embedding`)

In [ ]:
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

In [ ]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

In [14]:
VectorStoreIndex.from_documents?

Signature:
VectorStoreIndex.from_documents(
    documents: Sequence[llama_index.core.schema.Document],
    storage_context: Optional[llama_index.core.storage.storage_context.StorageContext] = None,
    show_progress: bool = False,
    callback_manager: Optional[llama_index.core.callbacks.base.CallbackManager] = None,
    transformations: Optional[List[llama_index.core.schema.TransformComponent]] = None,
    service_context: Optional[llama_index.core.service_context.ServiceContext] = None,
    **kwargs: Any,
) -> ~IndexType
Docstring:
Create index from documents.

Args:
    documents (Optional[Sequence[BaseDocument]]): List of documents to
        build the index from.
File:      /usr/local/lib/python3.11/site-packages/llama_index/core/indices/base.py
Type:      method

In [29]:
index = VectorStoreIndex.from_documents(documents)

Retrying llama_index.embeddings.openai.base.get_embeddings in 0.7422330702791277 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}.


KeyboardInterrupt: 

In [ ]:

query_engine = index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print(response)